## 调用示例

In [1]:

from transformers import pipeline#用人家设计好的流程完成一些简单的任务
classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/home/chongqinghuang/anaconda3/envs/waymo_38/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

## 1 Tokenizer

In [2]:
from transformers import AutoTokenizer#自动判断
 
model_name = "distilbert-base-uncased-finetuned-sst-2-english"  #根据这个模型所对应的来加载
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 输入文本
raw_inputs = [
    "I've been waiting for a this course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)
# 打印结果（得到两个字典映射：
# ‘input_ids’，一个tensor集合，每个词所对应的ID集合；
# attention_mask，一个tensor集合，表示是否是原来的词还是特殊字符等）：




tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

{'input_ids': tensor([[ 101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 2023, 2607, 2026, 2878,
         2166, 1012,  102],
        [ 101, 1045, 5223, 2023, 2061, 2172,  999,  102,    0,    0,    0,    0,
            0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}


In [3]:
# 如果想根据id重新获得原始句子，如下操作：
tokenizer.decode([ 101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 2023, 2607, 2026, 2878,2166, 1012,  102])

# 生成的文本会存在特殊字符，这些特殊字符是因为人家模型训练的时候就加入了这个东西，所以这里默认也加入了（google系的处理）
# "[CLS] i've been waiting for a this course my whole life. [SEP]"

"[CLS] i've been waiting for a this course my whole life. [SEP]"

## 2 模型的加载
模型的加载直接指定好名字即可（先不加输出层），这里model_name为模型名称（远程加载）或者本地路径（本地加载）。

In [4]:
from transformers import AutoModel
 
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(model_name )
model
# 打印出来模型架构，就是DistilBertModel（蒸馏后的bert模型，模型参数大约只有原来的60%，训练更快，但准确率下降不多）的架构了，能看到embeddings层、transformer层，看得还比较清晰：


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [5]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)


torch.Size([2, 15, 768])


## 3 模型基本逻辑
根据上面代码总结模型的逻辑：input—>词嵌入—>Transformer—>隐藏层—>Head层。

![](https://img-blog.csdnimg.cn/direct/e0724c93ace54f4c9eaf0064ea7114f7.png)

## 4 加入输出头

In [6]:
from transformers import AutoModelForSequenceClassification
 
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name )
outputs = model(**inputs)
print(outputs.logits.shape)


torch.Size([2, 2])


In [7]:
import torch
# 之后采用softmax进行预测：
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)


tensor([[1.5446e-02, 9.8455e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [8]:
# id2label这个我们后续可以自己设计，标签名字对应都可以自己指定：
model.config.id2label


{0: 'NEGATIVE', 1: 'POSITIVE'}